In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import pandas_ta as ta
import vectorbt as vbt
from tqdm import tqdm
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [20]:
import pickle
set_data = pickle.load(open('../data/set_data.bin', 'rb'))
set100_data_dict = pickle.load(open('../data/set100_data_dict.bin', 'rb'))

In [21]:
def custom_chop52ma(open, high, low, close, ma_period1, ma_period2, ma_period3, chop_length, symbols):
    vbt_ma1 = vbt.IndicatorFactory.from_pandas_ta('sma')
    vbt_ma2 = vbt.IndicatorFactory.from_pandas_ta('sma')
    vbt_ma3 = vbt.IndicatorFactory.from_pandas_ta('sma')
    vbt_chop = vbt.IndicatorFactory.from_pandas_ta('chop')

    week52_rolling = 252
    entries_result = {}
    exits_result = {}

    for i, symbol in enumerate(symbols):
        open_list = [item[i] for item in open]
        high_list = [item[i] for item in high]
        low_list = [item[i] for item in low]
        close_list = [item[i] for item in close]

        ma1 = vbt_ma1.run(close_list, length=ma_period1)
        ma2 = vbt_ma2.run(close_list, length=ma_period2)
        ma3 = vbt_ma3.run(close_list, length=ma_period3)
        chop = vbt_chop.run(close=close_list, low=low_list, high=high_list, length=chop_length)

        chop_df = pd.DataFrame(data=chop.chop.values, columns=['CHOP_'+str(chop_length)], index=chop.chop.index)
        ma1_df = pd.DataFrame(data=ma1.sma.values, columns=['SMA_'+str(ma_period1)], index=ma1.sma.index)
        ma2_df = pd.DataFrame(data=ma2.sma.values, columns=['SMA_'+str(ma_period2)], index=ma2.sma.index)
        ma3_df = pd.DataFrame(data=ma3.sma.values, columns=['SMA_'+str(ma_period3)], index=ma3.sma.index)

        _df = pd.concat([chop_df, ma1_df, ma2_df, ma3_df], axis=1)

        _df['open'] = open_list
        _df['high'] = high_list
        _df['low'] = low_list
        _df['close'] = close_list

        _df['above_ma'] = (_df['close'] > _df['SMA_'+str(ma_period1)]) & \
                            (_df['SMA_'+str(ma_period1)] > _df['SMA_'+str(ma_period2)]) & \
                            (_df['SMA_'+str(ma_period2)] > _df['SMA_'+str(ma_period3)])

        _df['52week_low'] = ((_df['close']/_df['low'].rolling(week52_rolling).min()) - 1)*100
        _df['52week_high'] = (1-(_df['close'] / _df['high'].rolling(week52_rolling).max())) * 100

        week52_high = 25
        _df['entries'] = (_df['above_ma'] == 1) & (_df['52week_high'] < week52_high) & \
                            (_df['CHOP_'+str(chop_length)] <= 38.2)
                        
        _df['exits'] = (_df['CHOP_'+str(chop_length)] >= 61.8)

        entries_result[f'{symbol}'] = _df['entries']
        exits_result[f'{symbol}'] = _df['exits']

    entries_result = pd.concat(entries_result, axis=1)
    exits_result = pd.concat(exits_result, axis=1)
    return entries_result, exits_result

In [22]:
def get_set100_df():
    sets100 = []
    for stock in set_data['Symbol']:
        sets100.append(stock)
    # set_tests = ['AOT', 'FORTH']
    master_df = {}

    open_df = pd.DataFrame()
    high_df = pd.DataFrame()
    low_df = pd.DataFrame()
    close_df = pd.DataFrame()

    for stock in sets100: # loop each stock
        stock_df = set100_data_dict[f'{stock}'].copy()
        stock_df = stock_df[['open', 'high', 'low', 'close']]

        open_df[f'{stock}'] = stock_df['open']
        high_df[f'{stock}'] = stock_df['high']
        low_df[f'{stock}'] = stock_df['low']
        close_df[f'{stock}'] = stock_df['close']

        # master_df[f'{stock}'] = stock_df[['open', 'high', 'low', 'close']]

    master_df['open'] = open_df
    master_df['high'] = high_df
    master_df['low'] = low_df
    master_df['close'] = close_df

    master_df = pd.concat(master_df, axis=1)
    master_df.columns.names=['Price', 'Ticker']
    return master_df

In [23]:
port_kwargs = dict(
    direction = 'long_only',
    freq = '1d',
    init_cash = 10000,
    fees = 0.0017,
    slippage = 0.000,
    size_type = 'Percent',
    size = 1
)

In [24]:
chop_52weekMA_indy = vbt.IndicatorFactory(class_name='Chop_52weekMA_indy',
                                        input_names=['open', 'high', 'low', 'close'],
                                        param_names=['ma_period1', 'ma_period2', 'ma_period3', 'chop_length'],
                                        output_names=['entries', 'exits'],      
                                        ).from_apply_func(custom_chop52ma)

In [7]:
set100_df = get_set100_df()
# set100_df = set100_df[0:260]
symbols = set100_df.loc[:, 'open'].columns.values
result = chop_52weekMA_indy.run(set100_df.open, set100_df.high, set100_df.low, set100_df.close,
            ma_period1=np.arange(30, 110, 10),
            ma_period2=np.arange(120, 190, 10),
            ma_period3=np.arange(200, 290, 10),
            chop_length=np.arange(10, 50, 5),
            symbols=symbols,
            # timestamp = set100_df.index,
            param_product=True)

In [8]:
result.entries.head(3)

chop_52weekma_indy_ma_period1      50                                     \
chop_52weekma_indy_ma_period2     150                                      
chop_52weekma_indy_ma_period3     200                                      
chop_52weekma_indy_chop_length     10                                      
Ticker                          DELTA    AOT    PTT   BDMS  KBANK  PTTEP   
datetime                                                                   
2018-01-19 09:00:00             False  False  False  False  False  False   
2018-01-22 09:00:00             False  False  False  False  False  False   
2018-01-23 09:00:00             False  False  False  False  False  False   

chop_52weekma_indy_ma_period1                               ...     60         \
chop_52weekma_indy_ma_period2                               ...    160          
chop_52weekma_indy_ma_period3                               ...    210          
chop_52weekma_indy_chop_length                              ...     20          
Ticker                          CPALL  PTTGC    BBL    WHA  ...  ASIAN   ITEL   
datetime                                                    ...                 
2018-01-19 09:00:00             False  False  False  False  ...  False  False   
2018-01-22 09:00:00             False  False  False  False  ...  False  False   
2018-01-23 09:00:00             False  False  False  False  ...  False  False   

chop_52weekma_indy_ma_period1                                             \
chop_52weekma_indy_ma_period2                                              
chop_52weekma_indy_ma_period3                                              
chop_52weekma_indy_chop_length                                             
Ticker                           INET     AS  WHART    PSH SABINA  MALEE   
datetime                                                                   
2018-01-19 09:00:00             False  False  False  False  False  False   
2018-01-22 09:00:00             False  False  False  False  False  False   
2018-01-23 09:00:00             False  False  False  False  False  False   

chop_52weekma_indy_ma_period1                 
chop_52weekma_indy_ma_period2                 
chop_52weekma_indy_ma_period3                 
chop_52weekma_indy_chop_length                
Ticker                            LPN    JKN  
datetime                                      
2018-01-19 09:00:00             False  False  
2018-01-22 09:00:00             False  False  
2018-01-23 09:00:00             False  False  

[3 rows x 2480 columns]

In [9]:
port = vbt.Portfolio.from_signals(set100_df.close,
                                entries = result.entries,
                                exits = result.exits,
                                **port_kwargs
                                )

In [10]:
port.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(50, 150, 200, 10, DELTA)",140.606040,2019-02-13 09:00:00,71.000000,16.971149,2019-03-14 09:00:00,70.500000,16.851634,-104.125803,-0.010430,Long,Closed,0
1,1,"(50, 150, 200, 10, DELTA)",123.488497,2019-04-04 09:00:00,80.000000,16.794436,2019-06-17 09:00:00,62.750000,13.173135,-2160.144145,-0.218658,Long,Closed,1
2,2,"(50, 150, 200, 10, DELTA)",129.248563,2020-07-07 09:00:00,59.750000,13.128423,2020-08-13 09:00:00,108.000000,23.730036,6199.384698,0.802759,Long,Closed,2
3,3,"(50, 150, 200, 10, DELTA)",94.958807,2020-09-18 09:00:00,146.500000,23.649491,2020-10-30 09:00:00,179.000000,28.895965,3033.615779,0.218066,Long,Closed,3
4,4,"(50, 150, 200, 10, DELTA)",87.771672,2020-11-24 09:00:00,193.000000,28.797885,2021-03-15 09:00:00,345.000000,51.478085,13261.018132,0.782826,Long,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7202,7202,"(60, 160, 210, 20, WHART)",837.895008,2019-02-07 09:00:00,11.914415,16.971149,2019-04-17 09:00:00,13.205143,18.809690,1045.713709,0.104749,Long,Closed,7202
7203,7203,"(60, 160, 210, 20, WHART)",669.046887,2019-06-21 09:00:00,16.481607,18.745845,2020-01-22 09:00:00,16.779468,19.084626,161.452503,0.014642,Long,Closed,7203
7204,7204,"(60, 160, 210, 20, PSH)",728.688237,2021-05-28 09:00:00,13.700000,16.971149,2021-09-15 09:00:00,13.200000,16.351764,-397.667032,-0.039834,Long,Closed,7204
7205,7205,"(60, 160, 210, 20, SABINA)",437.852162,2022-04-25 09:00:00,22.799999,16.971149,2022-05-24 09:00:00,23.299999,17.343323,184.611608,0.018493,Long,Closed,7205


In [12]:
comb_ret = port.total_return()
comb_ret

chop_52weekma_indy_ma_period1  chop_52weekma_indy_ma_period2  chop_52weekma_indy_ma_period3  chop_52weekma_indy_chop_length  Ticker
50                             150                            200                            10                              DELTA     0.539036
                                                                                                                             AOT      -0.132006
                                                                                                                             PTT      -0.021405
                                                                                                                             BDMS     -0.105387
                                                                                                                             KBANK    -0.057167
                                                                                                                                         ...   
60  

In [13]:
comb_ret.loc[comb_ret.values==comb_ret.max()]

chop_52weekma_indy_ma_period1  chop_52weekma_indy_ma_period2  chop_52weekma_indy_ma_period3  chop_52weekma_indy_chop_length  Ticker
50                             160                            200                            10                              XPG       12.523897
                                                              210                            10                              XPG       12.523897
Name: total_return, dtype: float64

In [14]:
port.stats()

Start                                 2018-01-19 09:00:00
End                                   2022-12-28 09:00:00
Period                                 1200 days 00:00:00
Start Value                                       10000.0
End Value                                    11958.373299
Total Return [%]                                19.583733
Benchmark Return [%]                             57.15876
Max Gross Exposure [%]                          90.403226
Total Fees Paid                                108.741993
Max Drawdown [%]                                22.466264
Max Drawdown Duration         430 days 14:12:18.626226584
Total Trades                                     2.906048
Total Closed Trades                              2.849597
Total Open Trades                                0.056452
Open Trade PnL                                 -103.55527
Win Rate [%]                                    50.804407
Best Trade [%]                                  27.863614
Worst Trade [%

In [ ]:
port.plot().show()

In [27]:
# trades_csv = trades_opt.copy()
trades_csv = port.trades.records_readable.copy()
ma1, ma2, ma3, chop, symbol = [], [] , [], [], []
for i, row in trades_csv.iterrows():
    ma1.append(row['Column'][0])
    ma2.append(row['Column'][1])
    ma3.append(row['Column'][2])

    chop.append(row['Column'][3])
    symbol.append(row['Column'][4])
    
trades_csv['ma1'] = ma1
trades_csv['ma2'] = ma2
trades_csv['ma3'] = ma3

trades_csv['chop'] = chop
trades_csv['symbol'] = symbol

In [28]:
trades_csv
# trades_csv.to_csv('chop52weekMA_result.csv',index=False)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id,ma1,ma2,ma3,chop,symbol
0,0,"(50, 150, 200, 10, DELTA)",140.606040,2019-02-13 09:00:00,71.000000,16.971149,2019-03-14 09:00:00,70.500000,16.851634,-104.125803,-0.010430,Long,Closed,0,50,150,200,10,DELTA
1,1,"(50, 150, 200, 10, DELTA)",123.488497,2019-04-04 09:00:00,80.000000,16.794436,2019-06-17 09:00:00,62.750000,13.173135,-2160.144145,-0.218658,Long,Closed,1,50,150,200,10,DELTA
2,2,"(50, 150, 200, 10, DELTA)",129.248563,2020-07-07 09:00:00,59.750000,13.128423,2020-08-13 09:00:00,108.000000,23.730036,6199.384698,0.802759,Long,Closed,2,50,150,200,10,DELTA
3,3,"(50, 150, 200, 10, DELTA)",94.958807,2020-09-18 09:00:00,146.500000,23.649491,2020-10-30 09:00:00,179.000000,28.895965,3033.615779,0.218066,Long,Closed,3,50,150,200,10,DELTA
4,4,"(50, 150, 200, 10, DELTA)",87.771672,2020-11-24 09:00:00,193.000000,28.797885,2021-03-15 09:00:00,345.000000,51.478085,13261.018132,0.782826,Long,Closed,4,50,150,200,10,DELTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7202,7202,"(60, 160, 210, 20, WHART)",837.895008,2019-02-07 09:00:00,11.914415,16.971149,2019-04-17 09:00:00,13.205143,18.809690,1045.713709,0.104749,Long,Closed,7202,60,160,210,20,WHART
7203,7203,"(60, 160, 210, 20, WHART)",669.046887,2019-06-21 09:00:00,16.481607,18.745845,2020-01-22 09:00:00,16.779468,19.084626,161.452503,0.014642,Long,Closed,7203,60,160,210,20,WHART
7204,7204,"(60, 160, 210, 20, PSH)",728.688237,2021-05-28 09:00:00,13.700000,16.971149,2021-09-15 09:00:00,13.200000,16.351764,-397.667032,-0.039834,Long,Closed,7204,60,160,210,20,PSH
7205,7205,"(60, 160, 210, 20, SABINA)",437.852162,2022-04-25 09:00:00,22.799999,16.971149,2022-05-24 09:00:00,23.299999,17.343323,184.611608,0.018493,Long,Closed,7205,60,160,210,20,SABINA
